### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv transformers accelerate
```

#### (2) .env 파일 세팅
```bash
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
성능 개선을 위한 테스트에서는 전체를 대상으로 인덱싱하지 않고, 5개 인덱싱 코드 실행 권장(주석 해제)

In [1]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import json

In [2]:
# device 자동 선택 (CUDA / MPS / CPU)
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(f"사용 디바이스: {device}")

사용 디바이스: cuda


In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
BASE_DIR = Path.cwd().parent  # /codeit-part3-team4
RAW_FOLDER = BASE_DIR / "data/raw/files"

# PDF to text
def extract_text(pdf_path: Path | str) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    return "\n".join(texts)

# 폴더에서 PDF 목록 가져오기
def get_pdf_paths(folder_path: Path | str) -> list[Path]:
    folder = Path(folder_path)
    pdf_paths = [p for p in folder.glob("*.pdf")]
    return sorted(pdf_paths)

In [5]:
# Chunking
def chunk(text: str, size: int = 800) -> list[str]:
    return [text[i:i+size] for i in range(0, len(text), size)]

In [6]:
# 임베딩 및 인덱스 만들기

# 한국어 임베딩 모델
embed_model_name = "nlpai-lab/KoE5"
embed_model = SentenceTransformer(embed_model_name, device=device)

def build_index(chunks: list[str]):
    embs = embed_model.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
    index = faiss.IndexFlatL2(embs.shape[1])
    index.add(embs.astype("float32"))
    return index, chunks

Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

print("모델 로딩 중")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id
)

모델 로딩 중


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'pad_token_id'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [8]:
# 데이터 폴더 설정
docs = get_pdf_paths(RAW_FOLDER)
print(f"발견된 PDF: {len(docs)}개")
for i, doc in enumerate(docs):
    print(f"{i}: {doc.name}")

발견된 PDF: 100개
0: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
1: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
2: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
3: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
4: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
6: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
7: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
8: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
9: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
10: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
11: 경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf
12: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
13: 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.pdf
14: 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
15: 광주과학기술원_학사시스템 기능개선 사업.pdf
16: 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
17: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
18: 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
19: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
20: 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
21: 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.pdf
22: 국방과학연구소_기록관리시스

In [9]:
# 전체 문서별 인덱스 저장
doc_indexes = {}
for doc_path in docs:
    print(f"처리 중: {doc_path.name}")
    text = extract_text(doc_path)
    chunks = chunk(text)
    index, chunks_list = build_index(chunks)
    doc_indexes[doc_path] = (index, chunks_list)
print("모든 문서 인덱싱 완료")

처리 중: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
처리 중: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
처리 중: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
처리 중: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
처리 중: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
처리 중: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
처리 중: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
처리 중: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
처리 중: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
처리 중: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
처리 중: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
처리 중: 경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf
처리 중: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
처리 중: 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.pdf
처리 중: 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
처리 중: 광주과학기술원_학사시스템 기능개선 사업.pdf
처리 중: 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
처리 중: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
처리 중: 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
처리 중: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
처리 중: 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
처리 중: 국민연금공단_사업장 사회보험료 지원 

In [ ]:
# # 전체 중 5개 문서만 문서별 인덱스 저장
# doc_indexes = {}
# for doc_path in docs[:5]:
#     print(f"처리 중: {doc_path.name}")
#     text = extract_text(doc_path)
#     chunks = chunk(text)
#     index, chunks_list = build_index(chunks)
#     doc_indexes[doc_path] = (index, chunks_list)
# print("5개 문서 인덱싱 완료")

In [10]:
# 질문 리스트 (key, 질문)
queries = [
    ("project_name", "사업(용역)명은 무엇인가?"),
    ("agency", "발주 기관(수요기관)은 어디인가?"),
    ("purpose", "사업 목적(추진 배경)은 무엇인가?"),
    ("budget", "총 사업 예산(사업비)은 얼마인가?"),
    ("contract_type", "계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?"),
    ("deadline", "입찰/제안서 제출 마감일시는 언제인가?"),
    ("duration", "사업 수행 기간은 얼마나 되는가?"),
    ("requirements_must", "필수 요구사항(기능/성능/보안 등)은 무엇인가?"),
    ("eval_items", "평가 항목(기술/가격 등) 구성은 어떻게 되는가?"),
    ("price_eval", "가격 평가 방식(최저가/협상 등)은 무엇인가?"),
    ("eligibility", "참가 자격 요건(면허/실적/인증/등급)은 무엇인가?"),
]

In [11]:
def build_query_prompt(queries):
    return "\n".join(
        f"{i}. {q}"
        for i, (_, q) in enumerate(queries, start=1)
    )

In [12]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

[분석 규칙]
- 추측 금지, 문서에 명시된 내용만 사용
- 문서에 없으면 반드시 "명시 없음"
- **출력은 질문 개수와 동일한 줄 수**
- **각 줄에는 답변 텍스트만 작성**
- 번호, 질문 문장, '답:', 기호, 설명을 절대 포함하지 말 것

[질문 목록]
{questions}

[컨텍스트]
{context}

질문 순서대로 답변만 한 줄씩 출력하라.
"""

In [13]:
import re

def answer(index, embed_model, chunks, queries, top_k: int = 15) -> dict:
    # 1. 검색용 대표 질문
    search_query = "RFP 주요 사업 정보 요약"

    q_emb = embed_model.encode([search_query], convert_to_numpy=True)
    _, I = index.search(q_emb.astype("float32"), top_k)

    context = "\n\n".join(chunks[i] for i in I[0])

    questions_text = build_query_prompt(queries)
    prompt = RFP_PROMPT.format(
        context=context,
        questions=questions_text
    )

    # 2. HuggingFace generator 호출
    output = generator(
        prompt,
        do_sample=False,
        temperature=0.0,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )[0]["generated_text"]

    # 3. 프롬프트 제거
    raw_answer = output[len(prompt):].strip()

    # 4. 불필요한 텍스트 정리
    cleaned_answers = []
    for line in raw_answer.splitlines():
        line = line.strip()
        if not line:
            continue

         # 깨진 문자 제거: �
        line = re.sub(r"[�]", "", line)

        # "1. 질문문?" 같은 줄 제거
        if re.match(r"^\d+\.\s+.*\?$", line):
            continue

        # "답:" 제거
        line = re.sub(r"^답\s*:\s*", "", line)

        # 질문 번호가 남아 있으면 제거
        line = re.sub(r"^\d+\.\s*", "", line)

        # 앞쪽 화살표 제거 (->, →)
        line = re.sub(r"^(->|→)\s*", "", line)

        # 깨진 문자 제거
        line = re.sub(r"[�]", "", line)

        # 질문/번호/답 제거
        line = re.sub(r"^답\s*[:：]\s*", "", line)
        line = re.sub(r"^\d+\.\s*", "", line)
        line = re.sub(r"^(->|→)\s*", "", line)

        # 따옴표 정리
        line = re.sub(r"^['\"]|['\"]$", "", line)

        # 없음 통일
        if line in ["없음", ""]:
            line = "명시 없음"
        if re.fullmatch(r"['\"]?명시\s*없음['\"]?", line):
            line = "명시 없음"

        cleaned_answers.append(line)

    # 5. key: value 매핑
    results = {}
    for (key, _), value in zip(queries, cleaned_answers):
        results[key] = value if value else "명시 없음"

    # 6. 답변 개수 부족 시 보정
    for key, _ in queries[len(cleaned_answers):]:
        results[key] = "명시 없음"

    return results

In [14]:
# 테스트: 단일 문서
if docs:
    test_doc = docs[0]
    print(f"\n=== {test_doc.name} 분석 ===")

    index, chunks = doc_indexes[test_doc]
    result = answer(index, embed_model, chunks, queries)

    for k, v in result.items():
        print(f"{k}: {v}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'temperature', 'repetition_penalty', 'eos_token_id', 'pad_token_id', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: 벤처확인종합관리시스템 기능 고도화 용역사업
agency: 중소벤처기업부
purpose: 벤처기업 육성을 위한 복수의결권 및 스톡옵션 시스템 고도화를 위한 사업
budget: 20억원 미만
contract_type: 제한경쟁
deadline: 명시 없음
duration: 명시 없음
requirements_must: 사용자 인증 기능, 회원 가입, 하자담보, 관리 방법론, 개발 장비, 통제 및 리스크 계획, 프로젝트 관리, 프로젝트 지원
eval_items: 기술
price_eval: 명시 없음
eligibility: 명시 없음


In [15]:
import json

results = {}

for doc in docs:
    print(f"\n=== {doc.name} 분석 ===")

    if doc not in doc_indexes:
        print(f"[SKIP] 인덱스 없음: {doc.name}")
        continue

    index, chunks = doc_indexes[doc]

    result = answer(
        index=index,
        embed_model=embed_model,
        chunks=chunks,
        queries=queries
    )

    results[doc.name] = result

    for k, v in result.items():
        print(f"{k}: {v}")

# JSON 저장
output_path = BASE_DIR / "rfp_answer_results.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n전체 문서 분석 완료! 저장 위치: {output_path}")


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


project_name: 벤처확인종합관리시스템 기능 고도화 용역사업
agency: 중소벤처기업부
purpose: 벤처기업 육성을 위한 복수의결권 및 스톡옵션 시스템 고도화를 위한 사업
budget: 20억원 미만
contract_type: 제한경쟁
deadline: 명시 없음
duration: 명시 없음
requirements_must: 사용자 인증 기능, 회원 가입, 하자담보, 관리 방법론, 개발 장비, 통제 및 리스크 계획, 프로젝트 관리, 프로젝트 지원
eval_items: 기술
price_eval: 명시 없음
eligibility: 명시 없음

=== (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf 분석 ===


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


project_name: BIFF&ACFM 온라인서비스 재개발 및 행사지원시스템 공급 용역
agency: 부산국제영화제
purpose: 웹 표준, 웹 접근성 및 웹 UI에 대한 기능개선을 통해 사용자 중심의 웹서비스를 제공하고, BIFF, ACFM에서 운영중인 웹사이트에 대하여, 당해 연도의 독창적인 디자인 UI 적용, 최신 트렌드를 반영하여 브랜드 가치 제고
budget: 243,000,000원(VAT 포함)
contract_type: 지역 제한, 협상에 의한 계약
deadline: 7. 2(화) 16:00까지
duration: 202
requirements_must: 명시 없음
eval_items: 명시 없음
price_eval: 명시 없음
eligibility: 명시 없음

=== (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf 분석 ===


...

project_name: 예약발매시스템 개량 ISMP 용역
agency: 한국철도공사
purpose: 예약발매 플랫폼 강화, 카카오, 네이버 등 타 플랫폼을 활용한 열차 승차권 예매 뿐만 아니라 해외 플랫폼과도 예약연동 확대, 외부 음성인식 AI 서비스와도 연계하여 철도 승차권 조회 및 예매가 가능하도록 예매 편의 개선
budget: 470 백만원(VAT 포함)
contract_type: 제한경쟁입찰 / 협상에 의한 계약
deadline: 명시 없음
duration: 체결일로부터 5개월 이내
requirements_must: 명시 없음
eval_items: 명시 없음
price_eval: 명시 없음
eligibility: 명시 없음

=== 한국한의학연구원_통합정보시스템 고도화 용역.pdf 분석 ===


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


project_name: 사업(용역)명은 통합정보시스템 고도화 용역입니다.
agency: 발주 기관(수요기관)은 한국한의학연구원이오.
purpose: 사업 목적(추진 배경)은 기관생명윤리위원회(IRB) 업무프로세스 구축, 동물실험계획 업무 프로세스 구축, 국가연구개발사업 연구개발비 사용기준 및 회계기준 준수, 기존 시스템 업무 프로세스 개선 및 기능 고도화를 위해서입니다.
budget: 총 사업 예산(사업비)는 140,000,000원(부가세 포함) 이내입니다.
contract_type: 계약 방식은 협상에 의한 계약(제한경쟁입찰)입니다.
deadline: 입찰/제안서 제출 마감일시는 언제인가
duration: 명시 없음
requirements_must: 명시 없음
eval_items: 명시 없음
price_eval: 명시 없음
eligibility: 명시 없음

=== 한국해양조사협회_2024년 항해용 간행물 품질관리 업무보조 시스템 구축.pdf 분석 ===


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


project_name: 디지털 품질검사 기능 개발 및 항해용 간행물 품질관리 업무 지원
agency: 한국해양조사협회
purpose: 해도제작, 해양정보 분야 등의 폭넓은 이해와 경험, 높은 전문성을 필요로 하는 사업으로, 참여 업체의 관련분야 사업수행 경험이 연구 결과의 완성도에 영향을 미친다고 판단되어 수행실적을 평가한다.
budget: 명시 없음
contract_type: 제한입찰(협상에 의한 계약)
deadline: 명시 없음
duration: 2024년 12월 20일까지
requirements_must: 명시 없음
eval_items: 명시 없음
price_eval: 명시 없음
eligibility: 명시 없음

=== 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.pdf 분석 ===
project_name: 한영대학교 특성화 맞춤형 교육환경 구축 – 트랙운영
agency: 한영대학교
purpose: 본 사업의 수행은 대학의 교육환경을 현대화시키고 학생들에게 더 나은 교육 경험을 제공하기 위함입니다.
budget: 미 명시
contract_type: 협상에 의한 계약
deadline: 15.(화)
duration: 미 명시
requirements_must: 시스템 운영 사양, 사용자 환경, 공통사항, 공통사항, 품질보증방안 수립, 개발방법론에 따른 산출물, 검수
eval_items: 명시 없음
price_eval: 명시 없음
eligibility: 명시 없음

전체 문서 분석 완료! 저장 위치: /home/spai0611/kmw/codeit-part3-team4/rfp_answer_results.json
